In [ ]:
# %% -*- coding: utf-8 -*-

""" Created on June 12, 2021 // @author: Sarah Shi """

# This code is a workaround for Google Colab to ensure that NumPy 1.26 is installed. 
# Note that it will make your Colab runtime restart, which is necessary for the changes to take effect.
# http://github.com/googlecolab/colabtools/issues/5115

import importlib.metadata
desired_version = "1.26.0"
try:
    installed_version = importlib.metadata.version("numpy")
    if installed_version == desired_version:
        print(f"NumPy {desired_version} is already installed.")
    else:
        print(f"Installing NumPy {desired_version} (current: {installed_version})...")
        !pip install numpy=={desired_version} --prefer-binary
        import os
        os._exit(00)  # Restart runtime for changes to take effect
except importlib.metadata.PackageNotFoundError:
    print(f"NumPy is not installed. Installing {desired_version}...")
    !pip install numpy=={desired_version} --prefer-binary
    import os
    os._exit(00)

# Import and load Python packages
import os
import numpy as np
import pandas as pd
!pip install PyIRoGlass
import PyIRoGlass as pig
from google.colab import files

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Load sample FTIR spectra and chemistry/thickness files for reference. 
# For downloading the file
if not os.path.exists('COLAB_BINDER.zip'):
    !wget https://github.com/sarahshi/PyIRoGlass/raw/main/Inputs/COLAB_BINDER.zip

# For unzipping the file
if not os.path.exists('COLAB_BINDER'):
    !unzip COLAB_BINDER.zip


We've loaded in sample data, with transmission and reflectance FTIR, from the GitHub repository for data format referencing. Examples workflows working with these spectra can be found on the [ReadTheDocs](https://pyiroglass.readthedocs.io/en/latest/). 

The Google Colab implementation here aims to get your transmission FTIR spectra processed. We remove degrees of freedom to simplify the process. 

The files necessary include all of your transmission FTIR spectra, as well as 'ChemThick.CSV' file, containing all chemistry and thickness data. Find an example [here](https://github.com/sarahshi/PyIRoGlass/blob/main/Inputs/COLAB_BINDER/Colab_Binder_ChemThick.csv). Ensure that all of your transmission FTIR spectra names (before the .CSV) match up identically with the sample names in the chemistry and thickness file. This is necessary to ensure that the algorithm can match up samples. 

# Load transmission FTIR spectra here: 

In [ ]:
# Specify the directory where you want to save the uploaded files
desired_spectra_directory = '/content/YOUR_DATA/TransmissionSpectra/'

# Create the directory if it doesn't exist
if not os.path.exists(desired_spectra_directory):
    os.makedirs(desired_spectra_directory)

# Change the current working directory
os.chdir(desired_spectra_directory)

# Upload the files
uploadediles = files.upload()


# Load ChemThick.CSV here:

In [ ]:
# Specify the directory where you want to save the uploaded files
desired_chemthick_directory = '/content/YOUR_DATA/'

# Change the current working directory
os.chdir(desired_chemthick_directory)

# Upload the files
uploadediles = files.upload()


# Run PyIRoGlass here:

In [ ]:
# Change back to the original directory (e.g., '/content')
os.chdir('/content/')

# Identify all transmission spectra CSVs
SPEC_PATH = os.path.join(os.getcwd(), desired_spectra_directory)

# Load chemistry and thickness file called Colab_Binder_ChemThick.CSV. If you name the file something else, modify it here.
CHEMTHICK_PATH = os.path.join(os.getcwd(), desired_chemthick_directory, 'Colab_Binder_ChemThick.csv')

# Set the output path name desired for your sample. We will call our output 'RESULTS'.
OUTPUT_PATH = 'RESULTS'

# Load a list of files and a dictionary of transmission FTIR spectra from the files identified earlier.
loader = pig.SampleDataLoader(SPEC_PATH, CHEMTHICK_PATH)
dfs_dict, chem, thick = loader.load_all_data()

# Run PyIRoGlass! df_output will be saved as a CSV called OUTPUT_PATH+'_DF.CSV' in the FINALDATA directory.
df_output, failures = pig.calculate_baselines(dfs_dict, OUTPUT_PATH)

# Default temperature and pressure conditions of analysis are set at 25 C and 1 bar. These are the defaults at LDEO.
# df_conc will be saved as a CSV called OUTPUT_PATH+'_H2OCO2.CSV' in the FINALDATA directory.
df_conc = pig.calculate_concentrations(df_output, chem, thick, OUTPUT_PATH)

# Display results here: 

In [ ]:
display(df_output)

In [ ]:
display(df_conc)